In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.datasets as datasets
import torchvision.transforms as transforms

In [2]:
class DropoutNet(nn.Module):
    def __init__(self):
        super(DropoutNet, self).__init__()
        self.fc1 = nn.Linear(784, 500)
        self.dropout = nn.Dropout(p=0.5)
        self.fc2 = nn.Linear(500, 10)
        
    def forward(self, x):
        x = x.view(x.size(0), -1)
        x = torch.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

In [3]:
class NonDropoutNet(nn.Module):
    def __init__(self):
        super(NonDropoutNet, self).__init__()
        self.fc1 = nn.Linear(784, 500)
        self.fc2 = nn.Linear(500, 10)
        
    def forward(self, x):
        x = x.view(x.size(0), -1)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [4]:
train_transform = transforms.Compose([
    #transforms.AutoAugment(),
    transforms.ToTensor(),
    transforms.Normalize((0.5,),(0.5,))
])

test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,),(0.5,))
])

train_set = datasets.MNIST(root='./data', train=True, download=False, transform=train_transform)
test_set = datasets.MNIST(root='./data', train=False, download=False, transform=test_transform)

train_loader = torch.utils.data.DataLoader(train_set, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=64)

In [5]:
dropout_model = DropoutNet()

dropout_criterion = nn.CrossEntropyLoss()
dropout_optimizer = optim.SGD(dropout_model.parameters(), lr=0.01)

In [6]:
for epoch in range(10):
    dropout_model.train()
    for imgs, labels in train_loader:
        dropout_optimizer.zero_grad()
        output = dropout_model(imgs)
        loss = dropout_criterion(output,labels)
        loss.backward()
        dropout_optimizer.step()
        
    dropout_model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for imgs, labels in test_loader:
            output = dropout_model(imgs)
            _, predicted = torch.max(output.data, 1)
            total += labels.size(0)
            correct += (predicted==labels).sum().item()
            
        print('드롭아웃 모델 정확도: {:.2f}%'.format(correct/total*100))

드롭아웃 모델 정확도: 89.09%
드롭아웃 모델 정확도: 91.20%
드롭아웃 모델 정확도: 92.00%
드롭아웃 모델 정확도: 92.83%
드롭아웃 모델 정확도: 93.31%
드롭아웃 모델 정확도: 94.04%
드롭아웃 모델 정확도: 94.39%
드롭아웃 모델 정확도: 94.77%
드롭아웃 모델 정확도: 94.95%
드롭아웃 모델 정확도: 95.29%


In [7]:
non_dropout_model = NonDropoutNet()

non_dropout_criterion = nn.CrossEntropyLoss()
non_dropout_optimizer = optim.SGD(non_dropout_model.parameters(), lr=0.01)

In [8]:
for epoch in range(10):
    non_dropout_model.train()
    for imgs, labels in train_loader:
        non_dropout_optimizer.zero_grad()
        output = non_dropout_model(imgs)
        loss = non_dropout_criterion(output,labels)
        loss.backward()
        non_dropout_optimizer.step()
        
    non_dropout_model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for imgs, labels in test_loader:
            output = non_dropout_model(imgs)
            _, predicted = torch.max(output.data, 1)
            total += labels.size(0)
            correct += (predicted==labels).sum().item()
            
        print('드롭아웃 미사용 모델 정확도: {:.2f}%'.format(correct/total*100))

드롭아웃 미사용 모델 정확도: 89.14%
드롭아웃 미사용 모델 정확도: 90.81%
드롭아웃 미사용 모델 정확도: 91.68%
드롭아웃 미사용 모델 정확도: 92.43%
드롭아웃 미사용 모델 정확도: 93.05%
드롭아웃 미사용 모델 정확도: 93.45%
드롭아웃 미사용 모델 정확도: 93.93%
드롭아웃 미사용 모델 정확도: 94.39%
드롭아웃 미사용 모델 정확도: 94.70%
드롭아웃 미사용 모델 정확도: 94.63%
